In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import os
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from itertools import cycle
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
import keras
from keras.utils import np_utils
from keras.models import Sequential,load_model
from keras.utils.vis_utils import plot_model
from keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D
import random
from scipy.interpolate import CubicSpline
%matplotlib inline
tf.__version__
#from keras.models import load_model

'2.10.0'

In [ ]:
def load_test():
    #Test ori data
    test_x = np.load(r"D:\Research Work\Thesis\New Data\Data\MIT Processed\Test Dataset\ecg_ori_all_reshaped_te_segments.npy")
    test_y = np.load(r"D:\Research Work\Thesis\New Data\Data\MIT Processed\Test Dataset\ecg_ori_all_te_labels.npy")

    print(test_x.shape)
    print(test_y)
    return test_x,test_y

In [ ]:
def load_train():
    #Test ori data
    train_x = np.load(r"D:\Research Work\Thesis\New Data\Data\MIT Processed\Train Dataset\Original\ecg_ori_all_reshaped_tr_segments.npy")
    train_y = np.load(r"D:\Research Work\Thesis\New Data\Data\MIT Processed\Train Dataset\Original\ecg_ori_all_tr_labels.npy")

    print(train_x.shape)
    print(train_y)
    return train_x,train_y

In [4]:
train_x,train_y = load_train()
test_x,test_y = load_test()
print(train_x.shape,train_y.shape,test_x.shape,test_y.shape)

(460, 1800, 1)
[1 3 0 0 3 2 2 1 2 0 3 0 2 1 3 3 2 0 0 0 0 0 0 0 1 2 0 0 0 2 3 0 0 1 1 0 0
 3 0 0 3 2 0 0 0 1 0 1 1 0 0 0 0 0 0 0 1 3 1 0 0 0 0 0 0 2 0 1 1 3 1 0 1 0
 0 0 1 3 1 0 2 0 2 1 1 2 2 1 0 1 1 2 0 2 0 0 0 2 2 3 2 0 1 0 1 0 0 0 0 0 0
 1 1 0 1 3 3 2 2 3 3 2 0 0 0 1 0 3 2 1 2 2 1 2 1 3 0 1 2 0 0 0 1 0 0 0 1 3
 0 0 0 1 0 0 0 3 1 1 1 2 1 1 2 0 0 0 0 1 0 3 0 0 3 2 0 1 3 3 2 1 0 0 0 0 2
 2 3 1 2 2 2 0 2 1 0 1 0 2 1 1 0 3 3 3 0 2 0 2 2 1 1 1 1 1 0 0 2 0 2 0 0 3
 1 1 3 0 1 1 0 0 0 3 0 0 1 0 0 0 2 3 0 2 0 2 0 3 2 1 0 1 0 3 3 0 0 0 2 2 0
 0 3 1 0 1 1 0 0 0 2 0 2 2 0 3 3 0 1 0 3 0 0 2 0 1 3 2 1 0 1 3 3 2 0 2 1 2
 2 0 0 0 2 0 2 1 2 0 0 1 2 3 0 2 0 2 3 1 0 3 2 3 1 0 3 1 0 1 0 3 0 0 1 2 1
 3 1 0 0 2 3 0 1 2 2 3 0 3 0 2 0 0 0 0 0 1 0 1 1 0 2 0 2 0 3 2 0 1 1 0 1 0
 3 2 1 3 0 1 0 0 0 0 3 3 0 0 2 0 2 1 0 0 2 3 3 2 2 2 2 0 2 0 1 2 0 3 2 0 0
 0 0 2 0 1 0 3 3 0 1 2 0 0 0 3 0 0 2 0 1 2 3 2 2 0 2 0 0 1 0 3 3 0 3 0 2 2
 1 2 0 0 3 1 3 0 3 1 3 0 3 0 3 2]
(192, 1800, 1)
[1 2 3 0 1 0 2 1 0 0 2 2 3 1 1 0 0 0

In [5]:
from collections import Counter

c = Counter(train_y)
print(c)

Counter({0: 198, 1: 95, 2: 94, 3: 73})


In [6]:
#Different Augmentation Functions
def no_augmentation(x):
    return x

# Jittering
def jitter(x, sigma=0.05):
    return x + np.random.normal(loc=0, scale=sigma, size=x.shape)

# Scaling Has issue with loading into memory. 
def scale(x, sigma=0.5):
    x_scaled = np.copy(x)
    for i in range(x_scaled.shape[0]):
        factor = np.random.normal(loc=1.0, scale=sigma)
        x_scaled[i] *= factor
    return x_scaled



# Flipping
def flip(x):
    # Check if data is 2D with a single feature
    if x.ndim == 3 and x.shape[2] == 1:
        return np.flip(x, axis=1)
    else:
        # Fallback for 1D data
        return np.flip(x, axis=0)


# Permutation
def permute(x, num_segments=10):
    split = np.array_split(x, num_segments, axis=0)
    np.random.shuffle(split)
    return np.concatenate(split, axis=0)

# Magnitude Warping
def magnitude_warp(x, sigma=0.001, num_knots=4):
    time_steps = np.linspace(0, 1, num=x.shape[0])
    knot_positions = np.linspace(0, 1, num=num_knots)
    knot_values = np.random.normal(loc=1.0, scale=sigma, size=num_knots)
    spline = CubicSpline(knot_positions, knot_values)
    warp_values = spline(time_steps)
    return list(x * warp_values)


# Rotate
def rotate(x):
   
    if x.ndim != 2:
        raise ValueError("Input data must be a 2D array.")

    
    num_features = x.shape[1]
    
    
    random_rotation_matrix = np.random.normal(size=(num_features, num_features))
    q, r = np.linalg.qr(random_rotation_matrix) 
    
    
    x_rotated = np.dot(x, q)
    
    return x_rotated


# Time Warping
def time_warp(x, sigma=0.2, num_knots=4):
    if x.ndim == 1:
        # For 1D data, we work directly with x
        original_steps = np.arange(len(x))
        new_steps = np.linspace(0, len(x) - 1, num=len(x))
    elif x.ndim == 2 and x.shape[1] == 1:
        # For 2D data with a single feature
        original_steps = np.arange(x.shape[0])
        new_steps = np.linspace(0, x.shape[0] - 1, num=x.shape[0])
    else:
        raise ValueError("Unsupported data shape for time_warp")

    knot_positions = np.linspace(0, len(new_steps) - 1, num=num_knots)
    knot_values = np.random.normal(loc=0.0, scale=sigma, size=num_knots) + knot_positions
    spline = CubicSpline(knot_positions, knot_values)
    warped_steps = spline(new_steps)
    warped_steps[warped_steps < 0] = 0
    warped_steps[warped_steps > len(new_steps) - 1] = len(new_steps) - 1

    return np.array([x[int(warped_step)] if x.ndim == 1 else x[int(warped_step), 0] for warped_step in warped_steps])


# Window Slicing
def window_slice(x, window_size=50):
    # Ensure the window size is smaller than the length of the data
    if window_size >= len(x):
        raise ValueError("Window size must be smaller than the length of the data")
    start = np.random.randint(0, len(x) - window_size)
    windowed_data = x[start:start + window_size]
    padding_length = len(x) - window_size
    padded_data = np.pad(windowed_data, (0, padding_length), mode='constant')
    return padded_data



# Window Warping
def window_warp(x, window_size=50, scale=2):
    if len(x) <= window_size:
        raise ValueError("The window size must be less than the size of the data.")
    start = random.randint(0, len(x) - window_size)
    windowed_data = x[start:start + window_size]
    warped_window = np.repeat(windowed_data, scale)
    remove_length = len(warped_window) - window_size
    if remove_length > 0:
        warped_window = warped_window[:-remove_length]  
    return np.concatenate([x[:start], warped_window, x[start + window_size:]])

In [7]:
aug_function = no_augmentation

def balance_and_augment(train_x, train_y, aug_function):
    #aug_function = globals()[function_name]
    c = Counter(train_y)
    m = max(c.values())
    aug_train, aug_y = [], []
 
    for k, v in c.items():
        val = m - v
        while val > 0:
            gen_idx = random.randrange(0, train_x.shape[0])
            if k != 3:  
                augmented_data = aug_function(train_x[gen_idx])
            else:
                aug_train.append(train_x[gen_idx])  
                
            if len(augmented_data)==1800:
                aug_train.append(augmented_data)
                aug_y.append(k)
            val -= 1
 
    print(np.array(aug_train).shape,train_x.shape)
    for i in range(len(train_x)):
        aug_train.append(train_x[i])
        aug_y.append(train_y[i])
 
    return np.array(aug_train), np.array(aug_y)
 
aug_train,aug_y = balance_and_augment(np.squeeze(train_x), train_y, aug_function)
aug_train = np.expand_dims(aug_train,axis=2)
c_ = Counter(aug_y)
print(c_,aug_train.shape)

(457, 1800) (460, 1800)
Counter({1: 198, 3: 198, 2: 198, 0: 198}) (917, 1800, 1)


In [8]:
#Hyper-parameters
BATCH_SIZE = 16
EPOCHS = 100
TIME_PERIODS = 1800 # The number of steps within one time segment
STEP_DISTANCE = 1800 # The steps to take from one segment to the next
#if step value is equal to TIME_PERIODS, then there is no overlap between the segments
LABELS = [0,1,2,3] #nsr, afib, pvc, lb
LABEL_CONDITION=['NSR','PVC','AFIB','LB']
num_classes = 4

In [9]:
print("\n--- Reshape data to be accepted by Keras ---\n")

train_samples, train_time_periods, train_sensors = train_x.shape[0],train_x.shape[1], train_x.shape[2]
test_samples,test_time_periods, test_sensors = test_x.shape[0], test_x.shape[1], test_x.shape[2]


train_input_shape = (train_time_periods*train_sensors)
train_x = (train_x.reshape(train_samples, train_time_periods)).astype("float64")
print('train_x shape: ', train_x.shape)
print('Train input shape:', train_input_shape)

test_input_shape = (test_time_periods*test_sensors)
test_x = (test_x.reshape(test_samples, test_time_periods)).astype("float64")
print('test_x shape: ', test_x.shape)
print('Test input shape:', test_input_shape)


#-----------------------------------------------------------------
# Convert type for Keras otherwise Keras cannot process the data
train_y = train_y.astype("float64")
print('train_y shape: ', train_y.shape)
test_y = test_y.astype("float64")
print('test_y shape: ', test_y.shape)


--- Reshape data to be accepted by Keras ---

train_x shape:  (460, 1800)
Train input shape: 1800
test_x shape:  (192, 1800)
Test input shape: 1800
train_y shape:  (460,)
test_y shape:  (192,)


In [10]:
def classifier():
    model = Sequential()
    #model.add(Reshape((time, features), input_shape=(input_shape,)))
    model.add(Conv1D(32, 18, name='conv0',activation='relu', input_shape=(1800,1)))
    model.add(Conv1D(32, 18, name='conv1',activation='relu'))
    model.add(Conv1D(64, 18, name='conv2',activation='relu'))
    model.add(Conv1D(64, 18, name='conv3',activation='relu'))
    model.add(Conv1D(128, 18, name='conv4',activation='relu'))
    model.add(Conv1D(128, 18, name='conv5',activation='relu'))
    model.add(Conv1D(256, 18, name='conv6',activation='relu'))
    model.add(Conv1D(256, 18, name='conv7',activation='relu'))
    model.add(MaxPooling1D(3,name='max1'))
    model.add(Conv1D(32, 18, name='conv8',activation='relu'))
    model.add(Conv1D(32, 18, name='conv9',activation='relu'))
    model.add(Conv1D(64, 18, name='conv10',activation='relu'))
    model.add(Conv1D(64, 18, name='conv11',activation='relu'))
    model.add(Conv1D(128, 18, name='conv12',activation='relu'))
    model.add(Conv1D(256, 18, name='conv13',activation='relu'))
    model.add(GlobalAveragePooling1D(name='gap1'))
    model.add(Dropout(.5,name='drop1'))
    model.add(Dense(4, name='dense1',activation='softmax'))
    
    return model

In [14]:
from keras.regularizers import l2
from keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D,BatchNormalization,Input,ELU

def build_improved_model():
    model = Sequential()

    # Convolutional block 1
    model.add(Conv1D(32, kernel_size=3, padding='same', input_shape=(1800, 1), kernel_regularizer=l2(0.001), name='conv0'))
    model.add(BatchNormalization(name='bn0'))
    model.add(ELU(name='elu0'))
    model.add(Dropout(0.2, name='dropout0'))

    # Convolutional block 2
    model.add(Conv1D(32, kernel_size=3, padding='same', kernel_regularizer=l2(0.001), name='conv1'))
    model.add(BatchNormalization(name='bn1'))
    model.add(ELU(name='elu1'))
    model.add(MaxPooling1D(pool_size=3, name='max0'))
    model.add(Dropout(0.2, name='dropout1'))

    # Convolutional block 3
    model.add(Conv1D(64, kernel_size=3, padding='same', kernel_regularizer=l2(0.001), name='conv2'))
    model.add(BatchNormalization(name='bn2'))
    model.add(ELU(name='elu2'))
    model.add(Dropout(0.3, name='dropout2'))

    # Convolutional block 4
    model.add(Conv1D(64, kernel_size=3, padding='same', kernel_regularizer=l2(0.001), name='conv3'))
    model.add(BatchNormalization(name='bn3'))
    model.add(ELU(name='elu3'))
    model.add(MaxPooling1D(pool_size=3, name='max1'))
    model.add(Dropout(0.3, name='dropout3'))

    # Convolutional block 5
    model.add(Conv1D(128, kernel_size=3, padding='same', kernel_regularizer=l2(0.001), name='conv4'))
    model.add(BatchNormalization(name='bn4'))
    model.add(ELU(name='elu4'))
    model.add(Dropout(0.4, name='dropout4'))

    # Convolutional block 6
    model.add(Conv1D(128, kernel_size=3, padding='same', kernel_regularizer=l2(0.001), name='conv5'))
    model.add(BatchNormalization(name='bn5'))
    model.add(ELU(name='elu5'))
    model.add(MaxPooling1D(pool_size=3, name='max2'))
    model.add(Dropout(0.4, name='dropout5'))

    # Flatten and Dense Layers
    model.add(Flatten())
    model.add(Dense(units=4, activation='softmax', kernel_regularizer=l2(0.001), name='dense_final'))

    return model


model = build_improved_model()
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv0 (Conv1D)              (None, 1800, 32)          128       
                                                                 
 bn0 (BatchNormalization)    (None, 1800, 32)          128       
                                                                 
 elu0 (ELU)                  (None, 1800, 32)          0         
                                                                 
 dropout0 (Dropout)          (None, 1800, 32)          0         
                                                                 
 conv1 (Conv1D)              (None, 1800, 32)          3104      
                                                                 
 bn1 (BatchNormalization)    (None, 1800, 32)          128       
                                                                 
 elu1 (ELU)                  (None, 1800, 32)         

In [15]:
batchsize = 32


ckpt_folder  = os.path.join(os.getcwd(), 'Aug_No_Aug_T2')
ckpt_path = os.path.join(ckpt_folder, 'No_Aug_Model_{epoch}')

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=ckpt_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,verbose=1)


model = build_improved_model()

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(decay=.01,learning_rate=0.001,beta_1=.009,beta_2=.8,epsilon=1e-08), metrics=[tf.keras.metrics.CategoricalAccuracy(
        name='accuracy'),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'),tf.keras.metrics.AUC(name='AUC')])
        

history = model.fit(train_x,np_utils.to_categorical(train_y, num_classes), epochs=150, batch_size = batchsize,
                   validation_data=(test_x,np_utils.to_categorical(test_y, num_classes)), callbacks=[model_checkpoint_callback])






Epoch 1/150
15/15 [==============================] - ETA: 0s - loss: 3.4400 - accuracy: 0.3152 - Recall: 0.2935 - Precision: 0.3261 - AUC: 0.5772
Epoch 1: val_accuracy improved from -inf to 0.37500, saving model to c:\Users\nbalasubramanian\Documents\Research Work\New Data\Code-20240119T232854Z-001\Code\Aug_No_Aug_T2\No_Aug_Model_1
15/15 [==============================] - 2s 42ms/step - loss: 3.4400 - accuracy: 0.3152 - Recall: 0.2935 - Precision: 0.3261 - AUC: 0.5772 - val_loss: 1.7514 - val_accuracy: 0.3750 - val_Recall: 0.1510 - val_Precision: 0.8529 - val_AUC: 0.6029
Epoch 2/150
15/15 [==============================] - ETA: 0s - loss: 2.9299 - accuracy: 0.3565 - Recall: 0.3261 - Precision: 0.3686 - AUC: 0.6197
Epoch 2: val_accuracy did not improve from 0.37500
15/15 [==============================] - 0s 10ms/step - loss: 2.9299 - accuracy: 0.3565 - Recall: 0.3261 - Precision: 0.3686 - AUC: 0.6197 - val_loss: 1.7988 - val_accuracy: 0.2812 - val_Recall: 0.0573 - val_Precision: 0.4231

In [16]:
train_y

array([1., 3., 0., 0., 3., 2., 2., 1., 2., 0., 3., 0., 2., 1., 3., 3., 2.,
       0., 0., 0., 0., 0., 0., 0., 1., 2., 0., 0., 0., 2., 3., 0., 0., 1.,
       1., 0., 0., 3., 0., 0., 3., 2., 0., 0., 0., 1., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 1., 3., 1., 0., 0., 0., 0., 0., 0., 2., 0., 1.,
       1., 3., 1., 0., 1., 0., 0., 0., 1., 3., 1., 0., 2., 0., 2., 1., 1.,
       2., 2., 1., 0., 1., 1., 2., 0., 2., 0., 0., 0., 2., 2., 3., 2., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 3., 3., 2., 2.,
       3., 3., 2., 0., 0., 0., 1., 0., 3., 2., 1., 2., 2., 1., 2., 1., 3.,
       0., 1., 2., 0., 0., 0., 1., 0., 0., 0., 1., 3., 0., 0., 0., 1., 0.,
       0., 0., 3., 1., 1., 1., 2., 1., 1., 2., 0., 0., 0., 0., 1., 0., 3.,
       0., 0., 3., 2., 0., 1., 3., 3., 2., 1., 0., 0., 0., 0., 2., 2., 3.,
       1., 2., 2., 2., 0., 2., 1., 0., 1., 0., 2., 1., 1., 0., 3., 3., 3.,
       0., 2., 0., 2., 2., 1., 1., 1., 1., 1., 0., 0., 2., 0., 2., 0., 0.,
       3., 1., 1., 3., 0.